In [69]:
# These settings automatically re-render the widget when the JavaScipt file has changed

%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [93]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle, IntSlider, Box, jslink
from nzshm_rupture_widget import rupture_map, MapWidget, SliderWidget, FullScreenWidget, HomeWidget
import json

In [94]:
with open('small_styled_crustal.geojson') as json_file:
    styled_crustal = json.load(json_file)
    json_file.close()

polygon = {"type":"FeatureCollection","features": [{
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [31.82, 9.27, 1],
                [31.82, 5.64, 1],
                [34.87, 5.64, 1],
                [34.87, 9.27, 1]
            ]
        ],
        "type": "Polygon"
    }
}]}


In [100]:
map = MapWidget(data=[styled_crustal, polygon], layout=Layout(grid_area='1 / 1 / -1 / -1'))
sliderWidget = SliderWidget(min=0, max=len(map.data)-1, value=0)
fullscreen = FullScreenWidget(value=0)
homeWidget = HomeWidget(map)

slider = IntSlider(min=0, max=len(map.data)-1, value=0)

jslink((map, "selection"), (sliderWidget, "value"))
jslink((map, "selection"), (slider, "value"))

right_bar = Box([sliderWidget], layout=Layout(
    display='flex',
    flex_flow='column',
    justify_content= 'flex-end',
   # border='solid 2px',
    align_items='stretch',
    width='100%',
    grid_area='2 / 3 / 4 / 4',
    pointer_events= 'none',
))

right_bar.add_class('bottomRight3DMap')

left_bar = Box([homeWidget, fullscreen], layout=Layout(
    display='flex',
    flex_flow='column',
    justify_content= 'flex-start',
   # border='solid 2px',
    align_items='stretch',
    width='100%',
    grid_area='1 / 1 / 4 / 2',
    pointer_events= 'none',
))

left_bar.add_class('bottomRight3DMap')

gridBox = GridBox(children=[map, right_bar, left_bar],
        layout=Layout(
            width='100%',
            height='400px',
            grid_template_rows='50px auto 50px',
            grid_template_columns='100px auto 300px')
       )
gridBox.add_class("fullScreenTarget")

gridBox



GridBox(children=(MapWidget(data=[{'type': 'FeatureCollection', 'features': [{'id': '216', 'type': 'Feature', …